In [430]:
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [431]:
sns.set(
    font='Times New Roman',
    font_scale=3.0,
    style='whitegrid',
    palette='dark'
)

In [432]:
DIR = os.getcwd()
SCHEDULER_PATHS = os.path.join(DIR, 'Schedulers', 'DynamicBehaviour')
SCHEDULERS = ['DistributedBlock', 'DistributedCTA', 'GlobalRoundRobin', 'Greedy', 'TwoLevelRoundRobin']
DIMENSIONS = 120

scheduler_dirs = os.listdir(SCHEDULER_PATHS)
scheduler_dirs = list(filter(lambda path: not 'DummyGPU' in path, scheduler_dirs))
scheduler_dirs = list(filter(lambda path: not path.startswith('.'), scheduler_dirs))
scheduler_dirs = list(map(lambda path: os.path.join(SCHEDULER_PATHS, path), scheduler_dirs))

In [433]:
#scheduler_dirs = [scheduler_dirs[0]]
#SCHEDULERS = [SCHEDULERS[0]]

In [434]:
def create_hist(df, filter=False):
    df_group = df.groupby(by=['x', 'y']).count()
    df_group = df_group.reset_index()
    df_group['occurrencies'] =  df_group['occurrencies'] / df_group['occurrencies'].max()
    if filter:
        df_group['occurrencies'][df_group['occurrencies'] >= 0.8] = np.nan
        df_group['occurrencies'] =  df_group['occurrencies'] * 100
    else:
        df_group['occurrencies'] =  np.log10((df_group['occurrencies']))
    # df_group['occurrencies'] =  (df_group['occurrencies'] / df_group['occurrencies'].max()) * 100.0
    df_group = df_group.sort_values(by=['x'])
    hist = df_group.reset_index().pivot(
        columns='y',
        index='x',
        values='occurrencies'
    )

    index = hist.index
    for idx in range(DIMENSIONS):
        if idx not in index:
            new_row = [np.nan for i in range(DIMENSIONS)]
            hist.loc[idx] = new_row

    hist = hist.transpose()
    return hist, df_group


def draw_hist(hist, scheduler):
    fig, ax = plt.subplots()
    #sns.heatmap(hist, cmap='magma', ax=ax)
    sns.heatmap(hist, cmap='cividis', ax=ax)
    ax.set(
        xlabel='X coordinates',
        ylabel='Y coordinates',
        title=scheduler
    )

    x_ticks = [tick * 16 for tick in range((DIMENSIONS // 16) + 1)]
    y_ticks = [tick * 16 for tick in range((DIMENSIONS // 16) + 1)]

    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_ticks)
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_ticks)

    ax.grid(False)

    return fig


def save_statistical_info(info, filename):
    with open(filename, 'w') as file:
        file.writelines(info)

In [435]:
for scheduler_dir in scheduler_dirs:
    for scheduler in SCHEDULERS:
        path_scheduler_csv = f'{scheduler_dir}/{scheduler}.csv'
        df = pd.read_csv(path_scheduler_csv)
        df['occurrencies'] = 1
        hist, group = create_hist(df)
        save_statistical_info(str(group.describe()), f'{scheduler_dir}/{scheduler}.txt')
        fig = draw_hist(hist, scheduler)
        fig.savefig(f'{scheduler_dir}/{scheduler}.png', format='png', bbox_inches='tight')
plt.close('all')

/var/folders/0s/1xw3l_x978315cbqp31k1mcm0000gn/T/ipykernel_44546/620636965.py:29: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


In [436]:
for scheduler_dir in scheduler_dirs:
    for scheduler in SCHEDULERS:
        path_scheduler_csv = f'{scheduler_dir}/{scheduler}.csv'
        df = pd.read_csv(path_scheduler_csv)
        df['occurrencies'] = 1
        hist, group = create_hist(df, True)
        save_statistical_info(str(group.describe()), f'{scheduler_dir}/{scheduler}_without100.txt')
        fig = draw_hist(hist, scheduler)
        fig.savefig(f'{scheduler_dir}/{scheduler}_without100.png', format='png', bbox_inches='tight')
plt.close('all')

/var/folders/0s/1xw3l_x978315cbqp31k1mcm0000gn/T/ipykernel_44546/620636965.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_group['occurrencies'][df_group['occurrencies'] >= 0.8] = np.nan
/var/folders/0s/1xw3l_x978315cbqp31k1mcm0000gn/T/ipykernel_44546/620636965.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_group['occurrencies'][df_group['occurrencies'] >= 0.8] = np.nan
/var/folders/0s/1xw3l_x978315cbqp31k1mcm0000gn/T/ipykernel_44546/620636965.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p